> Strategy 2

1/5 Minute Candles (EUR/UDS)

**Links**:

Library TA: https://technical-analysis-library-in-python.readthedocs.io/en/latest/index.html

API Data: https://www.alphavantage.co/documentation/

In [49]:
import pandas as pd

#df = pd.read_csv('1minEURUSD.csv', index_col = 0)

# Daily
import requests
from pprint import pprint
import pandas as pd
from datetime import date
import json

# Get data from Alpha Vantage 
#url = 'https://www.alphavantage.co/query?function=FX_DAILY&from_symbol=EUR&to_symbol=USD&apikey=9H93UG738ET21IS4'
symbol = 'EURUSD'
url = f"https://www.alphavantage.co/query?function=FX_DAILY&from_symbol={symbol[:3]}&to_symbol={symbol[3:]}&outputsize=full&apikey=9H93UG738ET21IS4"
r = requests.get(url)
data = r.json()['Time Series FX (Daily)']
df = pd.DataFrame.from_dict(data,orient='index').astype('float')

# Rename columns
df.rename(columns = {'1. open':'open', '2. high':'high', '3. low':'low', '4. close':'close',}, inplace = True)


#----

# Sort values by time (ascending)
df.sort_index(inplace=True)
df = df.loc['2021-01-01':'2022-12-31']

import ta

# Candle color
df['candleColor'] = df.apply(lambda row: 'GREEN' if row.close > row.open else 'RED' , axis=1)

# Candle body
df['candleBody'] = abs(df['open']-df['close'])

#RSI 
df['rsi'] = ta.momentum.RSIIndicator(df['close'], window=14, fillna=False).rsi()

# Bollinger Bands
df['bolHigh'] = ta.volatility.bollinger_hband(df['close'], window=21, window_dev=2, fillna=False)
df['bolLow'] = ta.volatility.bollinger_lband(df['close'], window=21, window_dev=2, fillna=False)
df['bolMid'] = ta.volatility.BollingerBands(df['close'], window=21, window_dev=2, fillna=False).bollinger_mavg()

# Skip first 21 rows used to calculate Bollinger bands
df = df.iloc[20:,:]

df = df.reset_index().rename(columns = {'index':'time'})

print(df)

           time     open     high     low    close candleColor  candleBody  \
0    2021-01-29  1.21200  1.21560  1.2092  1.21362       GREEN     0.00162   
1    2021-02-01  1.21285  1.21365  1.2053  1.20590         RED     0.00695   
2    2021-02-02  1.20590  1.20878  1.2010  1.20420         RED     0.00170   
3    2021-02-03  1.20432  1.20502  1.2002  1.20348         RED     0.00084   
4    2021-02-04  1.20351  1.20430  1.1956  1.19620         RED     0.00731   
..          ...      ...      ...     ...      ...         ...         ...   
345  2022-05-27  1.07292  1.07652  1.0695  1.07317       GREEN     0.00025   
346  2022-05-30  1.07326  1.07870  1.0724  1.07770       GREEN     0.00444   
347  2022-05-31  1.07762  1.07794  1.0677  1.07313         RED     0.00449   
348  2022-06-01  1.07308  1.07393  1.0625  1.06519         RED     0.00789   
349  2022-06-02  1.06531  1.07384  1.0643  1.07334       GREEN     0.00803   

           rsi   bolHigh    bolLow    bolMid  
0    45.614157  

In [53]:
# Algorithm
pip = 0.0001
lowRSI = 30
highRSI = 70

# Future data
future = df.iloc[1:,:]

# Position
position = 'Neutral'
price = 0
stopLoss = 0

# Stop Loss / Take Profit
slPips = pip*100

# Balance
p1 = 0
balance = 1000
trade = 0
perc = 1

# DATA FRAME DATA
d = {'time':[],'position':[],'price':[], 'previousClose':[], 'balance':[], 'stoploss':[]}

def updateDic(time,position,closePrice,index,balance,stopLoss,previousClose):
    # DataFrame data dictionary
    d['time'].append(time)
    d['position'].append(position)
    d['price'].append(closePrice)
    d['balance'].append(balance)
    d['stoploss'].append(stopLoss)
    d['previousClose'].append(previousClose)
    # Print
    #print(f"Time: {row['time']} -> {position} @ Price = {price} , PreviousClose = {previousClose}, balance = {balance}, stopLoss = {stopLoss}")

for index, row in future.iterrows():
    # Previous Update
    previous = df.iloc[index-1:index,:].to_dict('records')[0]

    if position == 'Neutral':
        # Go Long
        if row['close'] < row['bolLow'] and row['rsi'] < lowRSI:
            position = 'Long1'
            price = row['close']
            stopLoss = row['close'] - slPips

            p1 = row['close']
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue

        # Go Short
        elif row['close'] > row['bolHigh'] and row['rsi'] > highRSI:
            position = 'Short1'
            price = row['close']
            stopLoss = row['close'] + slPips

            p1 = row['close']
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
            
        # Mantain
        else:
            price = row['close']
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
    
    elif position == 'Long1':
        
        # If full change -> Neutral + Short1
        if row['close'] > row['bolHigh'] or row['rsi'] > highRSI:
            position = 'Short1'
            price = row['close']
            stopLoss = row['close'] + slPips
            
            balance = perc*balance*(row['close']-p1) + balance
            p1 = row['close']
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
            
        # Sell 1/2 of your position -> Long2
        elif row['close'] > row['bolMid']:
            position = 'Long2'
            price = row['close']
            
            # Update SL
            stopLoss = row['close'] - slPips
            
            trade = perc*balance*0.5
            balance = perc*balance*0.5*(row['close']-p1) + balance
            # no update in price - winnings are from the initial price
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
        
        
        # Check Stop Loss
        elif stopLoss > row['close']:
            position = 'Neutral'
            price = row['close']
            stopLoss = 0
            
            balance = perc*balance*(row['close']-p1) + balance
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
        
        # Update Trailing Stop Loss
        elif row['close'] > previous['close']: 
            stopLoss = row['close'] - slPips
            price = row['close']
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
        
        # Mantain
        else:
            price = row['close']
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
    
    elif position == 'Long2':
        
        # If change trend -> Short1
        if row['close'] > row['bolHigh'] or row['rsi'] > highRSI:
            position = 'Short1'
            price = row['close']
            stopLoss = row['close'] + slPips
            
            balance = trade*(row['close']-p1) + balance
            trade = 0
            
            p1 = row['close']
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
        
        # Check Stop Loss
        elif stopLoss > row['close']:
            position = 'Neutral'
            price = row['close']
            stopLoss = 0
            
            balance = trade*(row['close']-p1) + balance
            trade = 0
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
        
        # Update Trailing Stop Loss
        elif row['close'] > previous['close']: 
            stopLoss = row['close'] - slPips
            price = row['close']
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
        
        
        # Mantain
        else:
            price = row['close']
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
            
    elif position == 'Short1':
        
        # If full change -> Neutral + Long1
        if row['close'] < row['bolLow'] or row['rsi'] < lowRSI:
            position = 'Long1'
            price = row['close']
            stopLoss = row['close'] - slPips
            
            balance = perc*balance*(p1-row['close']) + balance
            p1 = row['close']
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
            
        # Sell 1/2 of your position -> Short2
        elif row['close'] < row['bolMid']:
            position = 'Short2'
            price = row['close']
            
            # Update SL
            stopLoss = row['close'] + slPips
            
            trade = perc*balance*0.5
            balance = perc*balance*0.5*(p1-row['close']) + balance
            # no update in price - winnings are from the initial price
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
        
        
        # Check Stop Loss
        elif stopLoss < row['close']:
            position = 'Neutral'
            price = row['close']
            stopLoss = 0
            
            balance = perc*balance*(p1-row['close']) + balance
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
        
        # Update Trailing Stop Loss
        elif row['close'] < previous['close']: 
            stopLoss = row['close'] + slPips
            price = row['close']
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
        
        # Mantain
        else:
            price = row['close']
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
    
    elif position == 'Short2':
        
        # If change trend -> Long1
        if row['close'] < row['bolLow'] or row['rsi'] < lowRSI:
            position = 'Long1'
            price = row['close']
            stopLoss = row['close'] - slPips
            
            balance = trade*(p1-row['close']) + balance
            trade = 0
            
            p1 = row['close']
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
        
        # Check Stop Loss
        elif stopLoss < row['close']:
            position = 'Neutral'
            price = row['close']
            stopLoss = 0
            
            balance = trade*(p1-row['close']) + balance
            trade = 0
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
        
        # Update Trailing Stop Loss
        elif row['close'] < previous['close']: 
            stopLoss = row['close'] + slPips
            price = row['close']
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
        
        
        # Mantain
        else:
            price = row['close']
            updateDic(row['time'],position,row['close'],index,balance,stopLoss,previous['close'])
            continue
    
        
        
print(balance)
dfTest = pd.DataFrame.from_dict(d)
dfTest.to_excel('testDailyS2nEURUSD.xlsx')

979.733361040041
